In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from tqdm import tqdm
# import os

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")


Using device: cuda


In [ ]:
# # Expecting:
# # train_loader
# # val_loader
# # class_names

# print("Classes:", class_names)


In [ ]:
# class ScratchMaskCNN(nn.Module):
#     def __init__(self):
#         super().__init__()

#         self.conv1 = nn.Sequential(
#             nn.Conv2d(3, 32, 3, padding=1),
#             nn.BatchNorm2d(32),
#             nn.ReLU(),
#             nn.MaxPool2d(2)
#         )

#         self.conv2 = nn.Sequential(
#             nn.Conv2d(32, 64, 3, padding=1),
#             nn.BatchNorm2d(64),
#             nn.ReLU(),
#             nn.MaxPool2d(2)
#         )

#         self.conv3 = nn.Sequential(
#             nn.Conv2d(64, 128, 3, padding=1),
#             nn.BatchNorm2d(128),
#             nn.ReLU(),
#             nn.MaxPool2d(2)
#         )

#         self.fc = nn.Sequential(
#             nn.Linear(128 * 16 * 16, 256),
#             nn.ReLU(),
#             nn.Dropout(0.5),
#             nn.Linear(256, 2)
#         )

#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.conv2(x)
#         x = self.conv3(x)
#         x = x.view(x.size(0), -1)
#         return self.fc(x)


In [ ]:
# model = ScratchMaskCNN().to(device)

# total_params = sum(p.numel() for p in model.parameters())
# print(f"Total parameters: {total_params:,}")


In [ ]:
# criterion = nn.CrossEntropyLoss()

# optimizer = optim.Adam(
#     model.parameters(),
#     lr=0.001
# )


In [ ]:
# def train_one_epoch(model, loader):
#     model.train()
#     running_loss = 0
#     correct = 0
#     total = 0

#     for images, labels in tqdm(loader, leave=False):
#         images, labels = images.to(device), labels.to(device)

#         optimizer.zero_grad()
#         outputs = model(images)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()
#         preds = outputs.argmax(1)
#         correct += (preds == labels).sum().item()
#         total += labels.size(0)

#     acc = correct / total
#     return running_loss / len(loader), acc


In [ ]:
# def validate(model, loader):
#     model.eval()
#     running_loss = 0
#     correct = 0
#     total = 0

#     with torch.no_grad():
#         for images, labels in loader:
#             images, labels = images.to(device), labels.to(device)

#             outputs = model(images)
#             loss = criterion(outputs, labels)

#             running_loss += loss.item()
#             preds = outputs.argmax(1)
#             correct += (preds == labels).sum().item()
#             total += labels.size(0)

#     acc = correct / total
#     return running_loss / len(loader), acc 


In [ ]:
# EPOCHS = 20
# best_val_acc = 0.0

# os.makedirs("Models", exist_ok=True)

# for epoch in range(EPOCHS):
#     train_loss, train_acc = train_one_epoch(model, train_loader)
#     val_loss, val_acc = validate(model, val_loader)

#     print(f"""
# Epoch [{epoch+1}/{EPOCHS}]
# Train Loss: {train_loss:.4f} | Train Acc: {train_acc*100:.2f}%
# Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc*100:.2f}%
# """)

#     # ⭐ SAVE FULL MODEL (NOT STATE_DICT)
#     if val_acc > best_val_acc:
#         best_val_acc = val_acc
#         torch.save(model, "Models/scratch_mask_cnn_best.pth")
#         print("✅ Best model saved!")


In [ ]:
# print(f"🏁 Training complete")
# print(f"Best Validation Accuracy: {best_val_acc*100:.2f}%")


# This is the main training loop 

In [ ]:
import sys
import os

# go one folder back to main project
sys.path.append("..")
import torch
import torch.nn as nn
import torch.optim as optim
import os
from tqdm import tqdm

from data_pipeline import get_dataloaders
from scratch_model import ScratchMaskCNN

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_loader, val_loader, class_names = get_dataloaders()
model = ScratchMaskCNN().to(device)


In [3]:
BATCH_SIZE = 32
IMG_SIZE = 128

train_loader, val_loader, class_names = get_dataloaders(
    dataset_path="Dataset",
    img_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    num_workers=0   # Windows-safe
)

print("Classes:", class_names)
print("Train batches:", len(train_loader))
print("Val batches:", len(val_loader))


Classes: ['with_mask', 'without_mask']
Train batches: 484
Val batches: 121


In [4]:
model = ScratchMaskCNN(num_classes=len(class_names)).to(device)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


ScratchMaskCNN(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (gap): AdaptiveAvgPool2d(output_size=(1, 1))
  (fc): Sequential(
    (0): Linear(in_features=128, out_featur

In [5]:
def train_one_epoch(model, loader):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total


In [6]:
def validate(model, loader):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total


In [7]:
EPOCHS = 20
best_val_acc = 0.0

os.makedirs("Models", exist_ok=True)

for epoch in range(EPOCHS):
    train_loss, train_acc = train_one_epoch(model, train_loader)
    val_loss, val_acc = validate(model, val_loader)

    print(f"""
Epoch [{epoch+1}/{EPOCHS}]
Train Loss: {train_loss:.4f} | Train Acc: {train_acc*100:.2f}%
Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc*100:.2f}%
""")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model, "Models/scratch_mask_cnn_best.pth")
        print("✅ Best model saved!")


Training:   7%|▋         | 32/484 [00:03<00:44, 10.26it/s]c:\Users\nikhi\AppData\Local\Programs\Python\Python311\Lib\site-packages\PIL\Image.py:1039: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(



Epoch [1/20]
Train Loss: 0.4260 | Train Acc: 81.04%
Val   Loss: 0.3721 | Val   Acc: 83.98%

✅ Best model saved!



Epoch [2/20]
Train Loss: 0.3510 | Train Acc: 85.09%
Val   Loss: 0.3270 | Val   Acc: 86.90%

✅ Best model saved!



Epoch [3/20]
Train Loss: 0.2831 | Train Acc: 88.60%
Val   Loss: 0.2229 | Val   Acc: 91.44%

✅ Best model saved!



Epoch [4/20]
Train Loss: 0.2433 | Train Acc: 90.39%
Val   Loss: 0.2058 | Val   Acc: 91.75%

✅ Best model saved!


...


Epoch [16/20]
Train Loss: 0.1244 | Train Acc: 95.38%
Val   Loss: 0.1116 | Val   Acc: 95.76%




Epoch [17/20]
Train Loss: 0.1184 | Train Acc: 95.80%
Val   Loss: 0.1074 | Val   Acc: 96.15%

✅ Best model saved!



Epoch [18/20]
Train Loss: 0.1156 | Train Acc: 95.44%
Val   Loss: 0.1838 | Val   Acc: 92.92%




Epoch [19/20]
Train Loss: 0.1155 | Train Acc: 95.81%
Val   Loss: 0.0976 | Val   Acc: 96.56%

✅ Best model saved!



Epoch [20/20]
Train Loss: 0.1135 | Train Acc: 95.86%
Val   Loss: 0.1086 | Val   Acc: 95.94%



In [8]:
print("🎉 Training completed!")
print(f"Best Validation Accuracy: {best_val_acc*100:.2f}%")


🎉 Training completed!
Best Validation Accuracy: 96.56%
